# **Recurrent Neural Networks - 필수 과제**

**LSTM**을 구현해봅시다!
<br><br><br>
**필요 사전 지식**:

- <u>PyTorch</u> (선택 과제 1)

<br>

**추가 사전 지식**: (알면 좋으나 몰라도 괜찮음)

- <u>Tokenization</u>, <u>Word Embedding</u> (선택 과제 2)

<br><br><br><br><br>

In [ ]:
!pip install transformers
!pip install datasets

In [2]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset

from tqdm import tqdm

<br><br>

[Hugging Face](https://huggingface.co)에서 [Rotten Tomatoes dataset](https://huggingface.co/datasets/rotten_tomatoes)과 [pretrained BERT](https://huggingface.co/bert-base-uncased)의 tokenizer를 가져오겠습니다.

또 학습 부담을 줄이기 위해 pretrained BERT에 내장된 word embedding layer의 weight도 가져옵시다.

In [3]:
# https://huggingface.co/datasets/rotten_tomatoes
dataset = load_dataset("rotten_tomatoes")

# https://huggingface.co/bert-base-uncased
pretrained_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
pretrained_embeddings = AutoModel.from_pretrained("bert-base-uncased").embeddings.word_embeddings

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

<br><br>

기본 BERT는 token을 768차원 벡터로 embedding합니다. 우리의 작은 dataset과 작은 모델에게 768차원은 부담스러우니 PCA를 사용해 64차원으로 줄여줍시다.

In [4]:
nano_embed = torch.pca_lowrank(pretrained_embeddings.weight.detach(), q=64)[0]

<br><br>

그런데 무작정 64차원으로 줄여도 되는 걸까요? BERT의 d_model이 괜히 768도 아닐 테고, 정보의 손실이 아주 클 것 같은데 말입니다.

궁금하니 코사인 유사도로 축소된 embedding layer에 token들의 정보가 그럭저럭 잘 남아있는지 확인해봅시다.

In [5]:
cos = (nano_embed @ nano_embed.T) / (nano_embed.abs() @ nano_embed.abs().T)

In [6]:
# word에 다양한 값을 넣어보세요! tokenizer의 vocab에 없는 token에 대해서는 빈 list가 뜹니다.
word = "jackson"

([*map(pretrained_tokenizer.decode, cos[pretrained_tokenizer.vocab[word]].argsort(descending=True)[1:21])] if word in pretrained_tokenizer.vocab else [])

['6',
 'all',
 'talked',
 'wanted',
 '50',
 'austin',
 'brighton',
 'local',
 'reached',
 'noah',
 'had',
 '"',
 'kurt',
 'pushed',
 'covered',
 'then',
 '25',
 'out',
 'talking',
 '5']

꽤 잘 남아있는 것 같습니다.

(TMI: 조금 더 욕심을 부려 한번 32차원으로 줄여보면 무시하기 어려운 정보의 손실을 체감할 수 있습니다.)

<br><br>

이제 LSTM을 구현합시다! 사실 원래 BiLSTM으로 하려고 했는데 underfitting이 심해서 그냥 plain LSTM으로 준비했습니다.

<br><br><br><br>
#### <span style="color:red">**<u>Q1.</u>**</span>

`class LSTMCell`의 빈칸을 채우세요.

In [7]:
class LSTMCell(nn.Module):
    def __init__(self, d_x, d_h): # d_x: x의 차원수 (scalar int)
                                  # d_h: h의 차원수 (scalar int)
        super().__init__()
        d_stack = d_x + d_h
        ######################### START OF YOUR CODE #########################

        dim1 = d_stack
        dim2 = d_h
        dim3 = d_stack
        dim4 = d_h
        dim5 = d_stack
        dim6 = d_h

        ########################## END OF YOUR CODE ##########################
        self.W_f = nn.Linear(d_stack, d_h)
        self.W_i = nn.Linear(dim1, dim2)
        self.W_C = nn.Linear(dim3, dim4)
        self.W_o = nn.Linear(dim5, dim6)


    # forward는 t-1의 h_{t-1}, C_{t-1}과 t의 x_t를 입력으로 받아 계산합니다.

    def forward(self, x, h, C): # x: x_t
                                # h: h_{h-1}
                                # C: C_{t-1}
        stack = torch.cat([x, h])
        ######################### START OF YOUR CODE #########################

        f = self.W_f(stack).sigmoid()
        i = self.W_i(stack).sigmoid()
        C_ = self.W_C(stack).tanh()

        C_t = f * C + i * C_

        o = self.W_o(stack).sigmoid()
        h_t = o * C_t.tanh()

        ########################## END OF YOUR CODE ##########################
        return h_t, C_t

In [8]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, d_out, pretrained_embeddings):
        super().__init__()
        vocab_size = pretrained_embeddings.shape[0]
        d_h = d_model = pretrained_embeddings.shape[1]

        self.embed = nn.Embedding(vocab_size, d_model, _weight=pretrained_embeddings.clone()) # word embedding layer
        self.cell = LSTMCell(d_x=d_model, d_h=d_h) # LSTM cell
        self.out = nn.Linear(d_h, d_out, bias=True) # output layer

        self.h_init = nn.Parameter(torch.zeros(d_h), requires_grad=False) # initial h
        self.C_init = nn.Parameter(torch.zeros(d_h), requires_grad=False) # initial C

    def forward(self, input_ids):
        embedded = self.embed(input_ids).squeeze()

        h = self.h_init.clone() # h 초기화
        C = self.C_init.clone() # C 초기화
        for x in embedded:
            h, C = self.cell(x, h, C) # iterate over embedded sequence

        return self.out(h).squeeze() # last hidden state를 output layer에 통과시킨 값을 반환

<br><br><br><br>
#### <span style="color:red">**<u>Q2.</u>**</span>

Test accuracy가 0.7 이상이 되도록 모델을 훈련시키세요.

In [9]:
######################### START OF YOUR CODE #########################

# 필요에 따라 바꿔도 됩니다.
device = "cuda"

########################## END OF YOUR CODE ##########################

model = LSTM(vocab_size=pretrained_tokenizer.vocab_size, d_out=1, pretrained_embeddings=nano_embed).to(device)

In [10]:
######################### START OF YOUR CODE #########################

# learning rate을 적절히 수정해보세요.
lr = 2.5e-04

########################## END OF YOUR CODE ##########################

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [11]:
train_loader = DataLoader(dataset["train"], shuffle=True)

In [12]:
######################### START OF YOUR CODE #########################

# 필요에 따라 바꿔도 됩니다.
num_print = 500
num_batch = 10

########################## END OF YOUR CODE ##########################


# train

save_l = 0
optimizer.zero_grad()
for i, data in enumerate(tqdm(train_loader)):
    text, label = data["text"][0], data["label"][0]
    input_ids = pretrained_tokenizer.encode(text, return_tensors="pt").to(device)
    y_pred = model(input_ids)

    label = label.to(device) * 1.
    loss = criterion(y_pred, label)
    loss.backward()

    if not (i+1)%num_batch:
        optimizer.step()
        optimizer.zero_grad()

    save_l += loss.item()
    if not (i+1)%num_print:
        print(f"{i+1:>5} iter: {save_l/num_print}")
        save_l = 0

  6%|▌         | 507/8530 [00:18<03:10, 42.18it/s]

  500 iter: 0.6934238826036453


 12%|█▏        | 1005/8530 [00:32<03:03, 40.90it/s]

 1000 iter: 0.69303952729702


 18%|█▊        | 1507/8530 [00:46<03:13, 36.25it/s]

...

 94%|█████████▍| 8003/8530 [03:52<00:21, 24.24it/s]

 8000 iter: 0.545979923978448


100%|█████████▉| 8504/8530 [04:06<00:01, 25.57it/s]

 8500 iter: 0.48562760627269747


100%|██████████| 8530/8530 [04:08<00:00, 34.39it/s]


In [13]:
test_loader = DataLoader(dataset["test"], shuffle=True)


# test

res = torch.tensor(0)
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        text, label = data["text"][0], data["label"][0]
        input_ids = pretrained_tokenizer.encode(text, return_tensors="pt").to(device)
        y_pred = model(input_ids)
        res += ((1 if y_pred > 0 else 0) == label)

print("Test accuracy:", res.item() / dataset["test"].num_rows)

100%|██████████| 1066/1066 [00:09<00:00, 115.24it/s]

Test accuracy: 0.7420262664165104


In [14]:
# 관찰용
# n 값을 바꿔가며 훈련시킨 모델의 예측값을 구경해보세요
n = 777

print(dataset["test"][n])
with torch.no_grad():
    print(model(pretrained_tokenizer.encode(dataset["test"][n]["text"], return_tensors="pt").to(device)).sigmoid().item())

{'text': "we started to wonder if \x85 some unpaid intern had just typed 'chris rock , ' 'anthony hopkins' and 'terrorists' into some univac-like script machine .", 'label': 0}
0.0919182077050209
